###**Import**

In [1]:
import numpy as np
import pandas as pd
import os, sys, copy
from itertools import product
from collections import defaultdict
from rdkit.Chem.rdmolfiles import MolFromSmiles
from rdkit.Chem.Graphs import CharacteristicPolynomial
from rdkit.Chem.rdmolops import AddHs, GetAdjacencyMatrix, GetDistanceMatrix

#-------------------------------------------------------------------------------

###**MMLToolkit**

In [2]:

def mae_power_error(y_true, y_HfP, y_MW, y_VMW, y_pred, save=False):
  return np.mean(np.abs((y_true-((y_pred-y_HfP)/y_MW*y_VMW))))

#-------------------------------------------------------------------------------

# MMLtoolkit.CV_tools.py
def mean_absolute_percentage_error(y_true, y_pred, save=False):
  return np.mean(np.abs((y_true-y_pred)/(y_true)))*100

def mean_absolute_error(y_true, y_pred, save=False):
  return np.mean(np.abs((y_true-y_pred)))

def neg_mean_absolute_error(y_true, y_pred, save=False):
  return -1*np.mean(np.abs((y_true-y_pred)))

#-------------------------------------------------------------------------------

# MMLtoolkit.featurizations.py
atom_num_dict = {'C':6,'N':7,'O':8,'H':1,'F':9, 'Cl': 17, 'S': 16 }

def bag_of_bonds(filename_list, verbose=False):
  num_mols = len(filename_list)
  # initialize empty dictionary for storing each bag as a list
  atom_types = ['C', 'N', 'O', 'F', 'H']
  num_atom_types = len(atom_types); empty_BoB_dict = {}
  # initialize empty list
  for atom_type in atom_types : empty_BoB_dict[atom_type] = []
  for i in range(num_atom_types):
    for j in range(i,num_atom_types): empty_BoB_dict[atom_types[i]+atom_types[j]]=[]
  # fill dicts in dict list
  BoB_dict_list = []
  if (verbose): print("creating intial BoBs")
  for m, filename in enumerate(filename_list):
    xyzfile = open(filename, 'r'); num_atoms_file = int(xyzfile.readline())
    xyzfile.close(); Cmat = np.zeros((num_atoms_file,num_atoms_file))
    chargearray = np.zeros((num_atoms_file, 1))
    xyzmatrix = np.loadtxt(filename, skiprows=2, usecols=[1,2,3])
    atom_symbols = np.loadtxt(filename, skiprows=2, dtype=bytes, usecols=[0])
    atom_symbols = [symbol.decode('utf-8') for symbol in atom_symbols]
    chargearray = [atom_num_dict[symbol] for symbol in atom_symbols]
    BoB_dict = copy.deepcopy(empty_BoB_dict)
    # populate BoB dict
    for i in range(num_atoms_file):
      for j in range(i, num_atoms_file):
        # concactenate to list
        if i == j : BoB_dict[atom_symbols[i]] += [0.5*chargearray[i]**2.4]
        else :
          dict_key = atom_symbols[i]+atom_symbols[j]
          dist=np.linalg.norm(xyzmatrix[i,:] - xyzmatrix[j,:])
          CM_term = chargearray[i]*chargearray[j]/dist
          # concactenate to list
          try : BoB_dict[dict_key] += [CM_term]
          except KeyError :
            dict_key = atom_symbols[j]+atom_symbols[i]
            BoB_dict[dict_key] += [CM_term]
    BoB_dict_list += [BoB_dict]
  '''
  tricky processing stage - zero pad all bags so they all have the same length
  and then cocatenate all bags into a feature vector for each molecule
  For each key in the dict, zero pad the bags, and do this for all molecules
  also sum these up to get the total length of the final feature vector
  '''
  feature_vect_length = 0
  if (verbose): print("finding max length of each bag and padding")
  for key in BoB_dict_list[0].keys():
    max_length = 0
    # find max bag length
    for i in range(num_mols):
      length = len(BoB_dict_list[i][key])
      if (length > max_length): max_length = length
    if (verbose): print(f'max length of {key} is {max_length}')
    # zero pad each bag
    for i in range(num_mols):
      pad_width = max_length - len(BoB_dict_list[i][key])
      BoB_dict_list[i][key] = BoB_dict_list[i][key]+[0]*pad_width
    feature_vect_length += max_length
  # initialize Numpy feature vector array
  X_BoB = np.zeros((num_mols, feature_vect_length))
  # concatenation of all bags
  if (verbose): print("concatenating the bags")
  for m in range(num_mols):
    featvec = []
    for key in BoB_dict_list[m].keys():
      featvec += sorted(BoB_dict_list[m][key], reverse=True)
    X_BoB[m,:] = np.array(featvec)
  # concatenate feature names
  feature_names = []
  for key in BoB_dict_list[0].keys():
    for element in BoB_dict_list[0][key]: feature_names += [key]
  return feature_names, X_BoB

def summed_bag_of_bonds(filename):
  """
  Based on Hansen, et al., The Journal of Physical Chemistry Letters 2015 6 (12), 2326-2331
  DOI: 10.1021/acs.jpclett.5b00831, URL: http://pubs.acs.org/doi/abs/10.1021/acs.jpclett.5b00831
    However, the Coulomb matrix terms for each atom pair (C-C, C-N, C-O, etc) are **summed** together.
    The diagonal terms of the Coulomb matrix are concatenated with the resulting vector.
    So the resulting feature vector for each molecule is a vector of length
    (num_atom_pair_types + num_atom_types). This is different than the original BoB, which maintains each
    CM entry in the feature vector.
  Args:
    filename : (string) the .xyz input filename for the molecule
  Returns:
    (feature_names, BoB_list) as lists
  """
  xyzfile = open(filename, 'r'); num_atoms_file = int(xyzfile.readline())
  xyzfile.close(); Cmat = np.zeros((num_atoms_file,num_atoms_file))
  chargearray = np.zeros((num_atoms_file, 1))
  xyzmatrix = np.loadtxt(filename, skiprows=2, usecols=[1,2,3])
  atom_symbols = np.loadtxt(filename, skiprows=2, dtype=bytes, usecols=[0])
  atom_symbols = [symbol.decode('utf-8') for symbol in atom_symbols]
  chargearray = [atom_num_dict[symbol] for symbol in atom_symbols]
  # Initialize dictionary for storing each bag
  atom_types = ['C', 'N', 'O', 'F', 'H']
  num_atom_types = len(atom_types); BoB_dict = {}
  for atom_type in atom_types: BoB_dict[atom_type] = 0
  for i in range(num_atom_types):
    for j in range(i,num_atom_types): BoB_dict[atom_types[i]+atom_types[j]] = 0
  # Populate BoB dict
  for i in range(num_atoms_file):
    for j in range(i, num_atoms_file):
      if i == j : BoB_dict[atom_symbols[i]] += 0.5*chargearray[i]**2.4
      else :
        dict_key = atom_symbols[i]+atom_symbols[j]
        dist=np.linalg.norm(xyzmatrix[i,:] - xyzmatrix[j,:])
        CM_term = chargearray[i]*chargearray[j]/dist
        try : BoB_dict[dict_key] += CM_term
        except KeyError :
          dict_key = atom_symbols[j]+atom_symbols[i]
          BoB_dict[dict_key] += CM_term
  # Process into list
  feature_names = list(BoB_dict.keys())
  BoB_list = [BoB_dict[feature] for feature in feature_names]
  return feature_names, BoB_list

def coulombmat_eigenvalues_from_coords(atom_types, coords, padded_size):
  """
  returns sorted Coulomb matrix eigenvalues
  Args:
    atom_types : a list of atom types (single characters)
    coords : the coords as a (num_atoms x 3) numpy array
    padded_size : the number of atoms in the biggest molecule to be considered
                  anything smaller will have zeros padded to the eigenvalue list
  Returns:
    Cmat_eigenvalues : as a Numpy array
  """
  atom_num_dict = {'C':6,'N':7,'O':8,'H':1,'F':9, 'Cl': 17, 'S': 16 }
  num_atoms = len(atom_types); Cmat = np.zeros((num_atoms,num_atoms))
  chargearray = np.zeros((num_atoms, 1))
  chargearray = [atom_num_dict[str(symbol,'utf-8')] for symbol in atom_types]
  for i, j in product(range(num_atoms), range(num_atoms)):
    if i == j : Cmat[i,j] = 0.5*chargearray[i]**2.4 # Diagonal terms
    else :
      dist = np.linalg.norm(coords[i,:] - coords[j,:])
      Cmat[i,j] = chargearray[i]*chargearray[j]/dist # Pair-wise repulsion
  Cmat_eigenvalues = np.linalg.eigvals(Cmat)
  # Cmat_eigenvalues = sorted(Cmat_eigenvalues, reverse=True) #sort (should be default)
  pad_width = padded_size - num_atoms
  Cmat_eigenvalues = np.pad(Cmat_eigenvalues, ((0, pad_width)), mode='constant')
  return Cmat_eigenvalues

def coulombmat_and_eigenvalues_as_vec(filename, padded_size, sort=True):
  """
  returns Coulomb matrix and **sorted** Coulomb matrix eigenvalues
  Args:
    filename : (string) the .xyz input filename for the molecule
    padded_size : the number of atoms in the biggest molecule to be considered (same as padded eigenvalue vector length)
  Returns:
    (Eigenvalues vector, Coulomb matrix vector) as Numpy arrays
  """
  xyzfile = open(filename, 'r'); num_atoms_file = int(xyzfile.readline())
  xyzfile.close(); Cmat = np.zeros((num_atoms_file,num_atoms_file))
  chargearray = np.zeros((num_atoms_file, 1))
  xyzmatrix = np.loadtxt(filename, skiprows=2, usecols=[1,2,3])
  atom_symbols = np.loadtxt(filename, skiprows=2, dtype=bytes, usecols=[0])
  atom_symbols = [symbol.decode('utf-8') for symbol in atom_symbols]
  chargearray = [atom_num_dict[symbol] for symbol in atom_symbols]
  for i, j in product(range(num_atoms_file), range(num_atoms_file)):
    if i == j : Cmat[i,j]=0.5*chargearray[i]**2.4 # Diagonal terms
    else :
      dist=np.linalg.norm(xyzmatrix[i,:] - xyzmatrix[j,:])
      Cmat[i,j]=chargearray[i]*chargearray[j]/dist # Pair-wise repulsion
  Cmat_eigenvalues = np.linalg.eigvals(Cmat)
  if (sort): Cmat_eigenvalues = sorted(Cmat_eigenvalues, reverse=True) # sort
  Cmat_as_vec = []
  for i, j in product(range(num_atoms_file), range(num_atoms_file)):
    if (j>=i): Cmat_as_vec += [Cmat[i,j]]
  pad_width = (padded_size**2 - padded_size)//2 + padded_size - ((num_atoms_file**2 - num_atoms_file)//2 + num_atoms_file)
  Cmat_as_vec = Cmat_as_vec + [0]*pad_width; Cmat_as_vec = np.array(Cmat_as_vec)
  pad_width = padded_size - num_atoms_file
  Cmat_eigenvalues = np.pad(Cmat_eigenvalues, ((0, pad_width)), mode='constant')
  return Cmat_eigenvalues, Cmat_as_vec

def literal_bag_of_bonds(mol_list, predefined_bond_types=[]):
  return sum_over_bonds(mol_list, predefined_bond_types=predefined_bond_types)

def sum_over_bonds(mol_list, predefined_bond_types=[], return_names=True):
  '''
    "Sum over bonds" aka "literal bag of bonds" aka "bond counting featurization"
    Note: Bond types are labeled according convention where the atom of the left is alphabetically less than
    the atom on the right. For instance, 'C=O' and 'O=C' bonds are lumped together under 'C=O', and NOT 'O=C'.
  Args:
    mol_list : a single mol object or list/iterable containing the RDKit mol objects for all of the molecules.
  Returns:
    bond_types : a list of strings describing the bond types in the feature vector
    X_LBoB : a NumPy array containing the feature vectors of shape (num_mols, num_bond_types)
  TODO: This code could be cleaned up substantially since we are using defaultdict now.
    <DCE 2018-06-12>
  '''
  if (isinstance(mol_list, list) == False): mol_list = [mol_list]
  empty_bond_dict = defaultdict(lambda : 0); num_mols = len(mol_list)
  if (len(predefined_bond_types) == 0 ):
    # first pass through to enumerate all bond types in all molecules ,
    # and set them equal to zero in the dict.
    for i, mol in enumerate(mol_list):
      bonds = mol.GetBonds()
      for bond in bonds:
        bond_start_atom = bond.GetBeginAtom().GetSymbol()
        bond_end_atom = bond.GetEndAtom().GetSymbol()
        bond_type = bond.GetSmarts(allBondsExplicit=True)
        bond_atoms = [bond_start_atom, bond_end_atom]
        if (bond_type == ''):
          bond_type = "-"
        bond_string = min(bond_atoms)+bond_type+max(bond_atoms)
        empty_bond_dict[bond_string] = 0
  else :
    for bond_string in predefined_bond_types:
      empty_bond_dict[bond_string] = 0
  # second pass through to construct X
  bond_types = list(empty_bond_dict.keys())
  num_bond_types = len(bond_types)
  X_LBoB = np.zeros([num_mols, num_bond_types])
  for i, mol in enumerate(mol_list):
    bonds = mol.GetBonds(); bond_dict = copy.deepcopy(empty_bond_dict)
    for bond in bonds:
      bond_start_atom = bond.GetBeginAtom().GetSymbol()
      bond_end_atom = bond.GetEndAtom().GetSymbol()
      # skip dummy atoms
      if (bond_start_atom=='*' or bond_end_atom=='*'): pass
      else :
        bond_type = bond.GetSmarts(allBondsExplicit=True)
        if (bond_type == ''): bond_type = "-"
        bond_atoms = [bond_start_atom, bond_end_atom]
        bond_string = min(bond_atoms)+bond_type+max(bond_atoms)
        bond_dict[bond_string] += 1
    # at the end, pick out only the relevant ones
    X_LBoB[i,:] = [bond_dict[bond_type] for bond_type in bond_types]
  if (return_names): return bond_types, X_LBoB
  else : return X_LBoB

def adjacency_matrix_eigenvalues(mol_list, useBO=False):
  eigenvalue_list = []; max_length = 0
  for mol in mol_list:
    adj_matrix = GetAdjacencyMatrix(mol, useBO=useBO)
    evs = list(np.linalg.eigvals(adj_matrix))
    # evs = sorted(evs, reverse=True)
    eigenvalue_list += [evs]; length = len(evs)
    if (length > max_length): max_length = length
  # zero padding
  for i in range(len(eigenvalue_list)):
    pad_width = max_length - len(eigenvalue_list[i])
    eigenvalue_list[i] += [0]*pad_width
  return np.array(eigenvalue_list)

def distance_matrix_eigenvalues(mol_list, invert=False):
  eigenvalue_list = []; max_length = 0
  for mol in mol_list:
    matrix = GetDistanceMatrix(mol)
    if (invert): matrix = np.nan_to_num(np.reciprocal(matrix))
    evs = np.linalg.eigvals(matrix); evs = np.real(evs)
    # evs = sorted(evs, reverse=True)
    eigenvalue_list += [evs]; length = len(evs)
    if (length > max_length): max_length = length
  # zero padding
  for i in range(len(eigenvalue_list)):
    pad_width = max_length - len(eigenvalue_list[i])
    eigenvalue_list[i] += [0]*pad_width
  return np.array(eigenvalue_list)

def characteristic_poly(mol_list, useBO=False):
  eigenvalue_list = []; max_length = 0
  for mol in mol_list:
    evs = CharacteristicPolynomial(mol, GetAdjacencyMatrix(mol, useBO=True))
    # evs = sorted(evs, reverse=True)
    eigenvalue_list += [list(evs)]; length = len(evs)
    if (length > max_length): max_length = length
  # zero padding
  for i in range(len(eigenvalue_list)):
    pad_width = max_length - len(eigenvalue_list[i])
    eigenvalue_list[i] += [0]*pad_width
  return np.array(eigenvalue_list)

#-------------------------------------------------------------------------------

###**Create feature**

In [3]:
# Remove extension
def b_mv(file_name):
  try :
    name_list = file_name.split('.')
    for i in ['xlsx', 'txt', 'zip']:
      if name_list[-1] == i : name_list.remove(i)
    return '.'.join(name_list)
  except : return file_name

#-------------------------------------------------------------------------------

def b_xlsx(warning=False, combine=False):
  global excel_name
  try :
    excel_name = b_mv(excel_name)
    df = pd.read_excel(f'{excel_name}.xlsx')
    # Combine every read-in sheet together.
    if combine == True :
      for i in range(0):
        name = f'descriptors.xls_{i+1}'
        sheet = pd.read_excel('descriptors.xls',sheet_name = name)
        df = df.join(sheet)
    # Delete column without name.
    try : df = df.drop(columns='Unnamed: 0')
    except : pass
  except : print(f'ERROR: {excel_name}.xlsx is not exist.\n'); sys.exit(0)
  return df
  try : df_name = df['Name']
  except :
    if warning == True : print("ERROR: the title of the column of name is not 'Name'.")
  try :
    df_smiles = df['SMILES']
    # Remove duplicate SMILES
    df_smiles_new = []
    for i in range(len(df_smiles)):
      if df_smiles[i] not in df_smiles_new :
        df_smiles_new.append(df_smiles[i])
      else :
        df = df.drop(labels=[i], axis=0)
        print(f'WARNING: smiles with the name {df_name[i]} is a duplicate,')
    if len(df_smiles) != len(df_smiles_new):
      print(f'Originally there were {len(df_smiles)} molecules,')
      print(f'and now there are {len(df_smiles_new)} molecules.\n'); sys.exit(0)
    else : print(f'No duplicate SMILES in {excel_name}.xlsx.')
  except :
    if warning == True : print("ERROR: the title of the column of SMILES is not 'SMILES'.")

#-------------------------------------------------------------------------------

def d_JPCL(a, keep, SoB=True, BoB=True):
  """
  a: Excel name
  keep: How many columns in front should be retained
  SoB, BoB: Create or not...
  """
  global excel_name
  excel_name = a; num_atoms = []
  df = b_xlsx(warning=True)
  data = df['SMILES'].apply(MolFromSmiles).apply(AddHs)
  # Calculate the number of atoms.
  for mol in data : mol = AddHs(mol); num_atoms += [mol.GetNumAtoms()]
  max_atoms = int(max(num_atoms))
  # For Coloumb Matrix
  while False :
    num_mols = len(data)
    X_Cmat_as_vec = np.zeros((num_mols, (max_atoms**2-max_atoms)//2 + max_atoms))
    X_Cmat_eigs = np.zeros((num_mols, max_atoms))
    X_Cmat_unsorted_eigs = np.zeros((num_mols, max_atoms))
  filename_list = []
  for i, refcode in enumerate(df['Name']):
    filename = f'{os.getcwd()}/xyz/{refcode}.xyz'
    try :
      no_such_file = 0
      # coulombmat_and_eigenvalues_as_vec >> returns Coulomb matrix and sorted it's eigenvalues.
      this_Cmat_eigs, this_Cmat_as_vec = coulombmat_and_eigenvalues_as_vec(filename, max_atoms)
      # There is 'sorted' on the top, and the default is turned off below, so it's unsorted.
      this_Cmat_unsorted_eigs, this_Cmat_as_vec = coulombmat_and_eigenvalues_as_vec(filename, max_atoms, sort=False)
      # Summed BoB
      summed_BoB_feature_names, summedBoB = summed_bag_of_bonds(filename)
      # Avoid repetitive execution.
      filename_list += [filename]
      # For Coloumb Matrix
      while False : X_Cmat_eigs[i,:] = this_Cmat_eigs
    except : no_such_file = 1
  if no_such_file == 1 :
    print('WARNING: upload the optimized xyz files to /xyz/ first.\n'); sys.exit(0)
  # Other Matrix
  while False :
    X_AM_eigs_BO = adjacency_matrix_eigenvalues(list(data), useBO=True)
    X_AM_eigs = adjacency_matrix_eigenvalues(list(data))
    X_DM_eigs = distance_matrix_eigenvalues(list(data))
    X_CP_BO = characteristic_poly(list(data), useBO=True)
    X_CP = characteristic_poly(list(data), useBO=False)
  # Save feature as excel
  df.drop(df.iloc[:,keep:], inplace=True, axis=1)
  if SoB == True :
    # X_LBoB >> number of bond_types, ex: H2O ['H-O'],[2]
    bond_types, X_LBoB = literal_bag_of_bonds(list(data))
    add_df = pd.DataFrame(X_LBoB, columns = bond_types)
    df.join(add_df).to_excel(f'{excel_name}_SoB.xlsx')
  if BoB == True :
    # Number of X_BoB > n(n-1)/2 with n = max number of atoms
    BoB_feature, X_BoB = bag_of_bonds(filename_list, verbose=False)
    add_df = pd.DataFrame(X_BoB, columns = BoB_feature)
    df.join(add_df).to_excel(f'{excel_name}_BoB.xlsx')

#-------------------------------------------------------------------------------

###**Main loop**

In [6]:
d_JPCL('Out', 8, SoB=True, BoB=True)
"""
a: Excel name
keep: How many columns in front should be retained
SoB, BoB: Create or not...
"""

SystemExit: 0